In [2]:
import pykep
import numpy as np

In [19]:

g = [0, 0, -3.7114];
cosphi = np.cos(pykep.DEG2RAD * 27);
α = 1 /(225 * 9.807 * cosphi);

T_max = 3100;
n = 6;
T1 = 0.3 * T_max;
T2 = 0.8 * T_max;
rho1 = n * T1 * cosphi;
rho2 = n * T2 * cosphi;

mwet = 1905;

r0 = (2000, 0, 1500) # x y z, unlike in paper BE CAREFUL
rdot0 = (100, 0, -75)

rfinal = [0, 0, 0]
rdotfinal = [0, 0, 0]

In [20]:
sc = pykep.sims_flanagan.spacecraft(mwet, 0.8, 225)
x0 = pykep.sims_flanagan.sc_state(r0, rdot0, mwet)
xf = pykep.sims_flanagan.sc_state(rfinal, rdotfinal, 0)

In [30]:
l = pykep.pontryagin.leg(sc = sc, mu=pykep.MU_EARTH, freemass=True, freetime=True, alpha=1, bound=True)
l.set(pykep.epoch(), x0, np.random.randn(7), pykep.epoch(1), xf)
l.mismatch_constraints(atol=1e-10, rtol=1e-10)

array([ 2.62592826e-13, -1.18083035e-34,  2.37776707e-13,  3.17672460e+03,
       -7.61024750e-19,  2.62010974e+03, -2.38223858e+00,  2.98352563e+17])

In [31]:
l.get_states()

C:\Users\rag\mambaforge\lib\site-packages\scipy\integrate\_ode.py:1181: UserWarning: dop853: larger nsteps is needed
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


array([[ 0.00000000e+00,  2.00000000e+03,  0.00000000e+00, ...,
         1.37349678e-01,  8.51020056e-01,  9.22234824e+09],
       [ 2.02951746e-16,  2.00000000e+03,  0.00000000e+00, ...,
         1.37349678e-01,  8.51020056e-01,  9.22234824e+09],
       [ 9.14664143e-16,  2.00000000e+03,  0.00000000e+00, ...,
         1.37349678e-01,  8.51020056e-01,  9.22234824e+09],
       ...,
       [ 8.04945703e-08,  6.71325820e-03, -7.27276668e-24, ...,
         5.46354828e-10, -6.80759582e-01,  5.63066850e+15],
       [ 8.04945703e-08,  7.05266013e-03, -7.44501676e-24, ...,
         5.83479319e-10, -6.79071124e-01,  5.63065616e+15],
       [ 8.04945703e-08,  7.40787945e-03, -7.62137699e-24, ...,
         6.23103122e-10, -6.77416969e-01,  5.63064498e+15]])

In [5]:
import pykep as pk
import pygmo as pg
import numpy as np
from matplotlib import pyplot as plt
from pykep.examples import add_gradient, algo_factory

# 1 - Algorithm
algo = algo_factory("slsqp") # slsqp ipopt

 # 2 - Problem
udp = add_gradient(pk.trajopt.indirect_pt2pl(
    x0=[44459220055.461708, -145448367557.6174, 1195278.0377499966,
        31208.214734303529, 9931.5012318647168, -437.07278242521573, 1000],
    t0=1285.6637861007277,
    pf="mars",
    thrust=0.1,
    isp=3000,
    mu=pk.MU_SUN,
    tof=[600, 720],
    alpha=0,    # quadratic control
    bound=True),
    with_grad=False
)
prob = pg.problem(udp)
prob.c_tol = [1e-5] * prob.get_nc()

# 3 - Population
pop = pg.population(prob)
z = np.hstack(([np.random.uniform(udp.udp_inner.tof[0],
                                    udp.udp_inner.tof[1])], 10 * np.random.randn(7)))
pop.push_back(z)

# 4 - Solve the problem (evolve)
pop = algo.evolve(pop)

homotopy_path = [0.2, 0.4, 0.6, 0.8, 0.9, 0.98, 0.99, 0.995, 1]
for alpha in homotopy_path:
    z = pop.champion_x
    print("alpha is: ", alpha)
    udp = add_gradient(pk.trajopt.indirect_pt2pl(
        x0=[44459220055.461708, -145448367557.6174, 1195278.0377499966,
            31208.214734303529, 9931.5012318647168, -437.07278242521573, 1000],
        t0=1285.6637861007277,
        pf="mars",
        thrust=0.1,
        isp=3000,
        mu=pk.MU_SUN,
        tof=[600, 720],
        alpha=alpha,    # quadratic control
        bound=True),
        with_grad=False
    )
    prob = pg.problem(udp)
    prob.c_tol = [1e-5] * prob.get_nc()

    # 7 - Solve it
    pop = pg.population(prob)
    pop.push_back(z)
    pop = algo.evolve(pop)

# 8 - Inspect the solution
print("Feasible?:", prob.feasibility_x(pop.champion_x))

# plot trajectory
axis = udp.udp_inner.plot_traj(pop.champion_x, quiver=True, mark='k')
plt.title("The trajectory in the heliocentric frame")

# plot control
udp.udp_inner.plot_control(pop.champion_x)
plt.title("The control profile (throttle)")

plt.ion()
plt.show()

ValueError: 
function: nlopt_objfun_wrapper
where: D:\bld\pagmo_1627978848243\work\src\algorithms\nlopt.cpp, 393
what: during an optimization with the NLopt algorithm 'slsqp' an objective function gradient was requested, but the optimisation problem '<class 'pykep.examples._ex_utilities.add_gradient'>' does not provide it


In [7]:
import pykep as pk
pk.examples.run_example7(solver = "ipopt")